# Segmenting and Clustering Neighborhoods in Toronto #

## Import Dependencies ##

In [ ]:
#%% Import Dependencies
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import wget #download files

print('Libraries imported.')

## Get Postal Codes Data from Wikipedia into DataFrame ##

In [ ]:
#%% Get Postal Codes Data
df0 = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df1=df0[0].copy()
df1.head()

## Group Neighbourhoods by Borough ##

In [ ]:
#%% Process Postal Codes Data
column_names=df1.iloc[0]
df1.columns=column_names
df1.drop(df1.index[0], inplace=True)

df1=df1[df1['Borough']!='Not assigned']
df1['Neighbourhood'][df1['Neighbourhood']=='Not assigned']=df1['Borough'][df1['Neighbourhood']=='Not assigned']

df2=df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)).to_frame()
df2.reset_index(inplace=True)

df2.shape

# Question 1: Postal Codes by Borough #

In [100]:
#Check Postal Code Data
df2.head(11)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Download Toronto Neighbourhood Geo Data##

In [ ]:
#%% Download Geo Data
url='https://cocl.us/Geospatial_data'
filename=wget.download(url)
filename

# Question 2: Postal Codes & Geo Data by Borough #
## Merge Download Toronto Postal Code and Geo Data##

In [101]:
#Merge Postal Code and Geo Data

df3=pd.read_csv(filename)
df2=pd.concat([df2,df3], axis=1)
df2.drop(['Postal Code'], axis=1, inplace=True)
df2.head(11)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Explode Neighbourhood Column into rows ##

In [ ]:
#%% Neighbourhood Info

df4 = pd.DataFrame(df2.Neighbourhood.str.split(', ').tolist()).stack().to_frame()
df4.reset_index(inplace=True)
df4[1]=df2['Borough'].iloc[df4['level_0']].values
df4[2]=df2['Latitude'].iloc[df4['level_0']].values
df4[3]=df2['Longitude'].iloc[df4['level_0']].values

df4.drop(['level_0','level_1'], axis=1, inplace=True)
df4.columns=['Neighbourhood','Borough','Latitude','Longitude']
df4.head()

## Limit Analysis to Downtown Toronto only ##

In [ ]:
df4=df4[df4['Borough']=='Downtown Toronto']
df4.head()
df4.shape

## Get Toronto Coordinates ##

In [ ]:
#%% Toronto Coordinates
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

## Create Toronto Test Map ##

In [ ]:
#%% Test Map

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

## Save Foursquare Login Info ##

In [ ]:
#Foursquare Login Info
CLIENT_ID = 'EH5NVS3H0UK5L4J43OQX4U5VSVIGPSGEOL5CAES4EEPABSBO' # your Foursquare ID
CLIENT_SECRET = '1L1UWLPCTNWWSEKDAVDLNPDI043MFYXR4WZA3PIO051KXO0V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## Define Function to get venues for each neighbourhood from Foursquare, Run Function ##

In [ ]:
#Function to get venues in each neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
LIMIT=100
radius=500
toronto_venues = getNearbyVenues(names=df4['Neighbourhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

## Group Venues to get List of Neighbourhoods ##
### Ensure no errors if a neighbourhood has no venues ###

In [ ]:
df5=toronto_venues.groupby('Neighbourhood').mean()
df5

In [ ]:
df5.drop(['Venue Latitude','Venue Longitude'], axis=1, inplace=True)
df5.reset_index(inplace=True)
print(df5.shape)
df5

## Perform One Hot Encoding for each Venue Category, Get Average per Neighbourhood ##

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move Neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

## K-Means Clustering on Average Dataset ##

In [ ]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

## Add Cluster Labels to list of neighbourhoods##

In [ ]:
# add clustering labels
df5.insert(0, 'Cluster Labels', kmeans.labels_)
df5.columns=['Cluster Labels','Neighbourhood','Latitude','Longitude']
df5

# Question 3: Downtown Toronto Clustered Neighbourhoods #
## Create cluster map of neighbourhoods in Folium and Display Map##

In [102]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df5['Latitude'], df5['Longitude'], df5['Neighbourhood'], df5['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters